# Trabalho prático de Análise Inteligente em Sistemas de "Big Data"

Importar bibliotecas

In [3]:
import pandas as pd

Juntar os datasets

In [6]:
# Read the data from the CSV files
ds1 = pd.read_excel('dataset_input/efeito_estufa.xlsx')
ds2 = pd.read_excel('dataset_input/emissoes_gases.xlsx')
ds3 = pd.read_excel('dataset_input/intensidade_carbonica.xlsx')
ds4 = pd.read_excel('dataset_input/PIBcrescimento.xlsx')
ds5 = pd.read_excel('dataset_input/PIBdespesas.xlsx')
ds6 = pd.read_excel('dataset_input/PIBproducao.xlsx')
ds7 = pd.read_excel('dataset_input/PIBrendimento.xlsx')

# Merge the data
dsAmbiente = pd.merge(pd.merge(ds1, ds2, on='Anos'), ds3, on='Anos')
dsPIB = pd.merge(pd.merge(ds4, ds5, on='Anos'), pd.merge(ds6, ds7, on='Anos'), on='Anos')

# verify joined datasets
dsPIB.dropna(inplace=True)
# drop repeated columns total, total_x and total_y are all the same values
dsPIB.drop(columns=['Total_x', 'Total_y'], inplace=True)

dsAmbiente.dropna(inplace=True)

print()

ds = pd.merge(dsAmbiente, dsPIB, on='Anos')

ds.to_csv('dataset_output/output.csv', index=False)

Index(['Anos', 'Total', 'Das atividades económicas', 'Das famílias',
       'Dióxido de carbono de origem fóssil',
       'Dióxido de carbono com origem na biomassa', 'Óxido nitroso', 'Metano',
       'Amoníaco', 'Compostos orgânicos voláteis não metanosos',
       'Monóxido de carbono', 'Óxidos de azoto', 'Óxidos de enxofre',
       'Hidrofluorcarbonetos', 'Hexafluoreto de enxofre',
       'Agricultura, produção animal, caça, floresta e pesca',
       'Indústrias extrativas', 'Indústrias transformadoras',
       'Eletridade, gás, vapor,...',
       'Captação, tratamento e distribuição de água...', 'Construção',
       'Comércio por grosso e a retalho,...', 'Transportes e armazenagem',
       'Alojamento, restauração e similares',
       'Atividades de informação e de comunicação',
       'Atividades financeiras e de seguros', 'Atividades imobiliárias',
       'Atividades de consultoria, científicas, técnicas e similares',
       'Atividades administrativas e dos serviços de apoio',
  